In [1]:
import gpytorch.kernels as kernel
import torch
import numpy as np
from src.test_statistics import *
from src.data import *
import random
import numpy as np
from src.test import *
import pandas as pd

/opt/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
X_ker = kernel.RBFKernel(ard_num_dims=3)
Y_ker = kernel.RBFKernel(ard_num_dims=1)

In [ ]:
kernel_X =ker(X_ker)

In [ ]:
X_ker.lengthscale = 10

In [ ]:
n=200
d=3
X = torch.randn((n,d))
Y = torch.randn(n)
T = (torch.randn((n))>0).float()

In [ ]:
from src.utils import compute_median_heuristic
from sklearn.metrics import pairwise_distances

In [ ]:
X_ker.lengthscale = compute_median_heuristic(X)
Y_ker.lengthscale = compute_median_heuristic(Y)

In [ ]:
Y_ker.lengthscale

In [ ]:
np.median(pairwise_distances(Y.reshape(-1,1)))

In [ ]:
compute_median_heuristic(X)
Y = Y.unsqueeze(0)
median_heuristic = [np.median(pairwise_distances(Y[:, [i]].reshape(-1,1))) for i in range(Y.shape[1])]
median_heuristic = median_heuristic[0]
Y =Y.squeeze(0)

In [ ]:
X.shape[0]

In [ ]:
a =torch.tensor([1,1],dtype=float) 

In [ ]:
data_train = Data_object(X,Y,T)
data_train_flip = Data_object(X,Y,1-T)

In [ ]:
m=200
X = torch.randn((m,d))
Y = torch.randn(m)
T = (torch.randn((m))>0).float()

In [ ]:
Y1 = torch.randn((1))

In [ ]:
data_test = Data_object(X,Y,T)
data_test_flip = Data_object(X,Y,1-T)

In [ ]:
W0 = get_W_matrix(X_ker(data_train.X0).evaluate(),1,func="cme")
W1 = get_W_matrix(X_ker(data_train.X1).evaluate(),1,func="cme")

In [ ]:
W0

In [ ]:
weights = 1/2*torch.ones(m)

In [ ]:
K = ker(X_ker)
L = ker(Y_ker)

In [ ]:
a = X_ker(X,X)

In [ ]:
Y_ker.lengthscale

In [ ]:
c = (data_test.T - weights)/(weights*(1-weights))
c

In [ ]:
data_train.Y.shape

In [ ]:
L(data_train.Y0,data_train.Y).shape

In [ ]:
DATE_test_stat(data_train,data_test,X_ker,Y_ker,weights,W0,W1)

In [ ]:
DETT_test_stat(data_train,data_test,X_ker,Y_ker,weights,W1)

In [ ]:
L(data_train.Y0,data_test.Y)@ c

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
weights_model = LogisticRegression()

In [ ]:
weights_model.fit(data_train.X,data_train.T)
weights_train = weights_model.predict_proba(data_train.X)[:,1]
weights_test = weights_model.predict_proba(data_test.X)[:,1]

In [ ]:
def DATE_goodness_of_fit(fit_samples,data_train,data_test,X_ker,Y_ker,weights,Wt,t=1):
        
        n = len(fit_samples)
        m = len(data_test.Y)

        K = ker(X_ker)
        L = ker(Y_ker)

        t_vec = (data_test.T == t).long()
        w = (t_vec)*1/(weights)

        fit_stat = 1/(n**2)*L(fit_samples,fit_samples).sum()
        fit_stat += -2/(n*m)*(w @ L(data_test.Y,fit_samples)).sum()
        fit_stat += -2/(n*m)*((L(fit_samples,data_train.Y1) @ (Wt @ (K(data_train.X1,data_test.X) @ (1-w))))).sum()
        fit_stat += 1/(m**2) * (w @ (L(data_test.Y,data_test.Y)@ w))
        fit_stat += 1/(m**2)*2 * (w @ (L(data_test.Y,data_train.Y1) @ (Wt @ (K(data_train.X1,data_test.X)@ (1-w)))))
        fit_stat += 1/(m**2)*(1-w @ (K(data_test.X,data_train.X1) @ (Wt @ ( L(data_train.Y1,data_train.Y1) @ (Wt @ (K(data_train.X1,data_test.X)@ (1-w)) ) ))))
        return fit_stat.item()


In [ ]:
from src.data import f_0,f_1
n_sample = 500

mu = [0,0,0,0,0,0,0,0,0,0]
sigma = ["C_ID",1.5]
noise = 0.2
g_0 = f_0
g_1 = f_1

In [ ]:
data_train = shift_data_simulation(mu,sigma,g_0,g_1,noise,1000)
data_test = shift_data_simulation(mu,sigma,g_0,g_1,noise,1000)
weights_model.fit(data_train.X,data_train.T)
weights_train = weights_model.predict_proba(data_train.X)[:,1]
weights_test = weights_model.predict_proba(data_test.X)[:,1]

In [ ]:
from src.utils import KMM_weights_for_W_matrix,kernel_mean_matching

In [ ]:
X_ker = kernel.RBFKernel(ard_num_dims=10)

In [ ]:
X_ker.lengthscale = 1

In [ ]:
X_ker.lengthscale

In [ ]:
data_train.X1[0]

In [ ]:
a = KMM_weights_for_W_matrix(X_ker,data_train.X1,data_test.X,True)

In [ ]:
from src.utils import get_W_matrix

In [ ]:
Wt = get_W_matrix(X_ker(data_train.X1).evaluate(),c=1,func="zero",weights=a)

In [ ]:
Wt.sum()

In [ ]:
from src.test_statistics import DATE_goodness_of_fit

In [ ]:
Wt = Wt.type(torch.float)
Wt.dtype

In [ ]:
DETT_goodness_of_fit(data_train.Y,data_train,data_test,X_ker,Y_ker,weights=torch.ones(data_train.X.shape[0]),t=1,Wt=Wt)

In [ ]:
from src.test import goodness_of_fit_test

In [ ]:
goodness_of_fit_test(data_train.Y,data_train,data_test,X_ker,Y_ker,weights_model,t=0,test_stat="DATE",reg=[1,1],func="zero", KMM_weights = False)

In [ ]:
goodness_of_fit_test(data_train.Y,data_train,data_test,X_ker,Y_ker,weights_model,t=0,test_stat="DATE",reg=[1,1],func="zero", KMM_weights = True)

In [ ]:
KMM_weights = True
reg =[1,1]
func ="zero"
fit_samples = data_train.Y1
weights = torch.tensor(weights_model.predict_proba(data_test.X)[:,1]).float()

In [ ]:
W0_weights_T = 1/KMM_weights_for_W_matrix(X_ker,data_train.X0,data_train.X,KMM_weights)            
W0_T = get_W_matrix(X_ker(data_train.X0).evaluate(),reg[0],func,weights=W0_weights_T)
DATE_goodness_of_fit(fit_samples,data_train,data_test,X_ker,Y_ker,1-weights,W0_T,t=1)

In [ ]:
KMM_weights = False

In [ ]:
fit_samples = data_train.Y0

In [ ]:
W0_weights = 1/KMM_weights_for_W_matrix(X_ker,data_train.X0,data_train.X,KMM_weights)     
W0 = get_W_matrix(X_ker(data_train.X0).evaluate(),reg[0],func,weights=W0_weights)

In [ ]:
kernel_X = ker(X_ker)
DATE_goodness_of_fit(fit_samples,data_train,data_test,X_ker,Y_ker,1-weights,W0,t=1)

In [ ]:
a,b,c = data_train.T,data_train.Y1,data_train.Y0

In [ ]:
data_train.flip_T()

In [ ]:
d,e,f = data_train.T,data_train.Y1,data_train.Y0

In [ ]:
data_train.flip_T()

In [ ]:
b = data_train.X1

In [ ]:
b

In [ ]:
# kernel_X = ker(X_ker)
DATE_goodness_of_fit(data_train.Y,data_train,data_test,X_ker,Y_ker,1-weights,W0,t=0)

In [ ]:
1-weights

In [ ]:
import numpy as np

In [ ]:
c =kernel_mean_matching_3(X_ker,(data_train.X1), (data_test.X),B=100,eps=10000)

In [ ]:
c.min(),c.max(),c.mean(),len(c)

In [ ]:
n0 = 3

In [ ]:
G = torch.concat([torch.ones(1,n0),-torch.ones(1,n0),torch.eye(n0),-torch.eye(n0)])
G

In [ ]:
eps,B = 0.5,2
h = torch.concat([torch.tensor([n0 * (1 + eps),n0 * (eps - 1)]), B * torch.ones((n0,)),torch.zeros(n0) ])
h

In [ ]:
import yaml
import src.data as data

In [ ]:
f = getattr(data,"shift_data_simulation")

In [ ]:
with open("/Users/jakefawkes/Python/DR_distributional_test/config/run_simulated_shift_experiment.yaml", "r") as f:
    cfg = yaml.safe_load(f)

In [ ]:
cfg

In [ ]:
def make_data(cfg):
    if cfg["data"]["generator"] == "shift_data_simulation":
        function_dict = {}
        for key in cfg["data"]["functions"]:
            function_dict[key] = getattr(data,cfg["data"]["functions"][key])
    data_train = shift_data_simulation(n_sample=cfg["data"]["n_train_sample"],**function_dict,**cfg["data"]["arguments"])
    data_test = shift_data_simulation(n_sample=cfg["data"]["n_test_sample"],**function_dict,**cfg["data"]["arguments"])
    return data_train,data_test

In [ ]:
a = shift_data_simulation(mu,sigma,g_0,g_1,noise,n_sample)

In [ ]:
float(True)

In [ ]:
import pandas as pd

In [ ]:
df_dict = {"Y":a.Y,"T":a.T}
df = pd.DataFrame(np.array(a.X))
df.columns = ["X"+str(i) for i in df.columns]

In [ ]:
df["Y"] = a.Y
df["T"] = a.T

In [ ]:
df

In [ ]:
import seaborn as sns

In [ ]:
b = sns.pairplot(df, hue ="T")

In [ ]:
from datetime import datetime


In [ ]:
now = datetime.now()

In [ ]:
now.strftime("%m-%d %H:%M:%S")

In [ ]:
from src.utils import cme_cross_validate_weighted,kernel_mean_matching

In [ ]:
cme_cross_validate_weighted(data_train,data_val,X_ker,Y_ker,reg_param=1,T_val=0)

In [ ]:
def cme_cross_validate_weighted(data_train,data_val,X_ker,Y_ker,reg_param,T_val=0):
        
        K = ker(X_ker)
        L = ker(Y_ker)

        X_train_data = data_train.X[data_train.T == T_val]
        Y_train_data = data_train.Y[data_train.T == T_val]

        W = get_W_matrix(X_ker(X_train_data).evaluate(),reg_param,"cme")

        X_val_data = data_val.X[data_val.T == T_val]
        Y_val_data = data_val.Y[data_val.T == T_val]

        return_weights = kernel_mean_matching(X_ker, X_val_data, data_val.X)
        return_weights_matrix = torch.diag(return_weights)

        val_stat = torch.trace(return_weights_matrix@(L(Y_val_data,Y_val_data) -2 * K(X_train_data,X_val_data).T @ (W @ (L(Y_train_data,Y_train_data))) + (K(X_train_data,X_val_data).T @ (W @ (L(Y_train_data,Y_train_data)@ (W @ (K(X_train_data,X_val_data))))))))
        return val_stat

In [ ]:
T_val = 0
reg_param = 1
X_train_data = data_train.X[data_train.T == T_val]
Y_train_data = data_train.Y[data_train.T == T_val]

W = get_W_matrix(X_ker(X_train_data).evaluate(),reg_param,"cme")

X_val_data = data_val.X[data_val.T == T_val]
Y_val_data = data_val.Y[data_val.T == T_val]

return_weights = kernel_mean_matching(X_ker, X_val_data, data_val.X).squeeze(0)
return_weights_matrix = torch.diag(return_weights)

val_stat = torch.trace(return_weights_matrix@(L(Y_val_data,Y_val_data) -2 * K(X_train_data,X_val_data).T @ (W @ (L(Y_train_data,Y_val_data))) + (K(X_train_data,X_val_data).T @ (W @ (L(Y_train_data,Y_train_data)@ (W @ (K(X_train_data,X_val_data))))))))

In [ ]:
(K(X_train_data,X_val_data).T @ (W @ (L(Y_train_data,Y_train_data)@ (W @ (K(X_train_data,X_val_data)))))).shape

In [ ]:
(K(X_train_data,X_val_data).T @ (W @ (L(Y_train_data,Y_val_data)))).shape

In [ ]:
torch.diag(return_weights.squeeze(0))

In [ ]:
data_val.T.sum()

In [ ]:
result_dict = kernel_permutation_test(data_train,data_test,X_ker,Y_ker,weights_model,test_stat="DATE",n_bins=10,n_permutations=100,reg=1,KMM_weights=True)

In [ ]:
len(data_val.T)

In [ ]:
X_val_data = data_val.X[data_val.T == 0]

In [ ]:
kernel_mean_matching(X_ker,data_test.X1,data_test.X)

In [ ]:
data_train.X

In [ ]:
K = ker(X_ker)
L = ker(Y_ker)
W0 = get_W_matrix(X_ker(data_train.X0).evaluate(),1,"cme")
W1 = get_W_matrix(X_ker(data_train.X1).evaluate(),1,"cme")

In [ ]:
L(data_val.Y0,data_val.Y0) -2 * K(data_train.X0,data_val.X0).T @ (W0 @ (L(data_train.Y0,data_val.Y0)))+ (K(data_train.X0,data_val.X0).T @ (W0 @ (L(data_train.Y0,data_train.Y0)@ (W0 @ (K(data_train.X0,data_val.X0))))))

In [ ]:
# Load config file
with open("config/wrapped_simulaton_cfg.yaml", "r") as f:
    cfg = yaml.safe_load(f)

In [ ]:
cfg

In [ ]:
import doubleml as dml

In [ ]:
df = data_train.pd_df()

In [ ]:
X_cols = [col for col in df.columns if col.startswith("X")]

In [ ]:
dml_data = dml.DoubleMLData(df,y_col="Y",d_cols="T",x_cols = X_cols)

In [ ]:
from sklearn.base import clone

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV

learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 5)

ml_l_bonus = clone(learner)

ml_m_bonus = clone(learner)

learner = LassoCV()

ml_l_sim = clone(learner)

ml_m_sim = clone(learner)
obj_dml_plr_bonus = dml.DoubleMLPLR(dml_data, ml_l_bonus, ml_m_bonus)

In [ ]:
obj_dml_plr_bonus.fit()

In [ ]:
obj_dml_plr_bonus.pval

In [ ]:
from src.comparison_models import tmle_test

In [ ]:
tmle_test(data_test)

In [ ]:
import os
import yaml
import seaborn as sns

In [ ]:
path_to_dir = "/Users/jakefawkes/Python/DR_distributional_test/sandbox/08-08 09:57:19"
os.chdir(path_to_dir)

In [ ]:
os.getcwd()

In [ ]:
with open("scores.metrics", "r") as f:
    results_dict = yaml.safe_load(f)
with open("cfg.yaml", "r") as f:
    cfg = yaml.safe_load(f)

In [ ]:
results_dict["result"] = [int(p_val <0.05) for p_val in results_dict["p_val"]]

In [ ]:
results_df = pd.DataFrame(results_dict)

In [ ]:
results_df

In [ ]:
if cfg["moving_param"]["beta_scalar"]:
    moving_param = "beta_scalar"
if cfg["moving_param"]["n_train_sample"]:
    moving_param = "n_sample"

In [ ]:
plot = sns.lineplot(data = results_df,x=moving_param,y="result",hue = "test_stat")

In [ ]:
fig = plot.get_figure()

In [ ]:
fig.savefig("results_plot")

In [ ]:
def load_IDHP():
    data = pd.read_csv("https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/IHDP/csv/ihdp_npci_1.csv", header = None)
    col =  ["treatment", "y_factual", "y_cfactual", "mu0", "mu1" ,]
    for i in range(1,26):
        col.append("x"+str(i))
    data.columns = col
    data = data.astype({"treatment":'float'}, copy=False)
    data
    X = torch.tensor(data[["x"+str(i) for  i in range(1,26)]].values)
    Y = torch.tensor(data["y_factual"])
    T = torch.tensor(data["treatment"])
    Y_cf = torch.tensor(data["y_cfactual"])
    return X,Y,T,Y_cf


        

In [ ]:
from src.data import IDHP_data_object

In [ ]:
data_train, data_test, data_full = IDHP_data_object(null_hypothesis = False)

In [ ]:
weights_model = LogisticRegression()
X_ker = kernel.RBFKernel(ard_num_dims=25)
Y_ker = kernel.RBFKernel(ard_num_dims=1)

In [ ]:
weights_model = LogisticRegression().fit(data_train.X,data_train.T)

In [ ]:
weights_train = torch.tensor(weights_model.predict_proba(data_train.X)[:,1]).float()
weights_test = torch.tensor(weights_model.predict_proba(data_test.X)[:,1]).float()


W0_weights = 1/KMM_weights_for_W_matrix(X_ker,data_train.X0,data_train.X,KMM_weights)
W1_weights = 1/KMM_weights_for_W_matrix(X_ker,data_train.X1,data_train.X,KMM_weights)

W0 = get_W_matrix(X_ker(data_train.X0).evaluate(),reg[0],func,weights=W0_weights)
W1 = get_W_matrix(X_ker(data_train.X1).evaluate(),reg[1],func,weights=W1_weights)

base_stat = DATE_test_stat(data_train,data_test,X_ker,Y_ker,weights_test,W0,W1)

In [ ]:
base_stat

In [ ]:
result = kernel_permutation_test(data_train,data_test,X_ker,Y_ker,weights_model,test_stat="DATE",n_bins=10,n_permutations=200,reg=[1,1],permute_weights=False,func="cme", KMM_weights = False)

In [ ]:
[result["stat"]]

In [ ]:
a.mean()

In [ ]:
import yaml

In [ ]:
import tqdm

In [ ]:
with open("config/real_data_experiment.yaml", "r") as f:
    cfg = yaml.safe_load(f)

In [ ]:
result_dict = {"test_stat":[], "p_val":[],"result":[],"base_stat":[]}
for i in tqdm.tqdm(range(cfg["experiment"]["n_iter"])):
        
        if cfg["experiment"]["dataset"] == "IDHP":
            data_train,data_test, data_full = IDHP_data_object(cfg["experiment"]["null_hypothesis"])
            
        weights_model.fit(X= data_train.X, y=data_train.T)
        
        for stat in cfg["experiment"]["test_stat"]:
            for func in cfg["experiment"]["ker_regress"]:
                result = kernel_permutation_test(data_train,data_test,X_ker,Y_ker,weights_model,test_stat=stat,n_bins =10,permute_weights=True , reg=[1,1],func = func,KMM_weights = False)
                result_dict["test_stat"] += [stat+func]
                result_dict["p_val"] += [result["p_val"].item()]
                result_dict["base_stat"] += [result["stat"]]
                result_dict["result"] += [int(result["p_val"].item()<0.05)]
        
        for model in cfg["experiment"]["comparison_model"]:
            result = comparison_model_dict[model](data_full)
            result_dict["test_stat"] += [model]
            result_dict["p_val"] += [result.item()]
            result_dict["base_stat"] += [0]
            result_dict["result"] += [int(result.item()<0.05)]


In [ ]:
result_dict

In [ ]:
X_ker.lengthscale = compute_median_heuristic(data_full.X)

In [ ]:
a =X_ker.lengthscale

In [ ]:
a[a==0] = 1

In [ ]:
X_ker.lengthscale = a

In [ ]:
X_ker.lengthscale

In [ ]:
X_ker.lengthscale

In [ ]:
data_full.X[:,23]

In [ ]:
result_dict = {"test_stat":[], "p_val":[],"result":[],"base_stat":[]}
if cfg["experiment"]["dataset"] == "IDHP":
    dx = 25
X_ker = kernel_dict[cfg["experiment"]["X_ker"]](ard_num_dims = 25)
Y_ker = kernel_dict[cfg["experiment"]["Y_ker"]](ard_num_dims=1)
n_bins = cfg["experiment"]["n_bins"]
permute_weights = cfg["experiment"]["permute_weights"] 
KMM_weights = cfg["experiment"]["KMM_weights"] 

if permute_weights:
    print("Permuting")
if not permute_weights:
    print("Not Permuting")
cme_reg = cfg["experiment"]["cme_reg"]

if cfg["experiment"]["dataset"] == "IDHP":
    data_train,data_val, data_full = IDHP_data_object(cfg["experiment"]["null_hypothesis"])

X_ker.lengthscale = compute_median_heuristic(data_full.X)
Y_ker.lengthscale = compute_median_heuristic(data_full.Y)
if type(cme_reg) is list:
    cross_val = True
    reg_param_range = np.linspace(cme_reg[0],cme_reg[1],num=cme_reg[2])  
    cme_reg_0 = cme_cross_validate_weighted(data_train,data_val,X_ker,Y_ker,reg_param_range,T_val=0)
    cme_reg_1 = cme_cross_validate_weighted(data_train,data_val,X_ker,Y_ker,reg_param_range,T_val=1)
    cme_reg = [cme_reg_0,cme_reg_1]
else: 
    cross_val = False
    cme_reg = [cme_reg,cme_reg]

for i in tqdm.tqdm(range(cfg["experiment"]["n_iter"])):
    
    if cfg["experiment"]["dataset"] == "IDHP":
        data_train,data_test, data_full = IDHP_data_object(cfg["experiment"]["null_hypothesis"])
        
    weights_model.fit(X= data_train.X, y=data_train.T)
    
    for stat in cfg["experiment"]["test_stat"]:
        for func in cfg["experiment"]["ker_regress"]:
            result = kernel_permutation_test(data_train,data_test,X_ker,Y_ker,weights_model,test_stat=stat,n_bins =n_bins,permute_weights=permute_weights , reg=cme_reg,func = func,KMM_weights = KMM_weights)
            result_dict["test_stat"] += [stat+func]
            result_dict["p_val"] += [result["p_val"].item()]
            result_dict["base_stat"] += [result["stat"]]
            result_dict["result"] += [int(result["p_val"].item()<0.05)]
    
    for model in cfg["experiment"]["comparison_model"]:
        result = comparison_model_dict[model](data_full)
        result_dict["test_stat"] += [model]
        result_dict["p_val"] += [result.item()]
        result_dict["base_stat"] += [0]
        result_dict["result"] += [int(result.item()<0.05)]


In [2]:
import pandas as pd

In [ ]:
def load_LBIDD(size = "1k"):
    X_data = pd.read_csv("data/x.csv")
    Y_data = pd.read_csv("data/"+size+"_f.csv")
    Y_CF_data = pd.read_csv("data/"+size+"_f.csv")
    data = Y_data.merge(X_data,on="sample_id")
    data = Y_CF_data.merge(data,on="sample_id")
    Y = torch.tensor(data["yf"],dtype=torch.float)
    T = torch.tensor(data["T"],dtype=torch.float)
    Y_cf = torch.tensor(data["y_cf"],dtype=torch.float)
    return X,Y,T,Y_cf

In [7]:
size = "1k"
X_data = pd.read_csv("data/x.csv")
Y_data = pd.read_csv("data/"+size+"_f.csv")
Y_CF_data = pd.read_csv("data/"+size+"_cf.csv")
data = Y_data.merge(X_data,on="sample_id")
data = Y_CF_data.merge(data,on="sample_id")

In [8]:
data[["y0","y1"]]

,y0,y1
0,12.506389,11.707886
1,12.994761,12.196257
2,12.941845,12.143341
3,13.443833,12.645329
4,13.713150,12.914646
...,...,...
995,13.834394,13.035891
996,13.618665,12.820162
997,12.863620,12.065117
998,12.898318,12.099815


In [23]:
def LBIDD_data_object(size = "1k",null_hypothesis = False):
    X_data = pd.read_csv("data/x.csv")
    Y_data = pd.read_csv("data/"+size+"_f.csv")
    Y_CF_data = pd.read_csv("data/"+size+"_cf.csv")
    data = Y_data.merge(X_data,on="sample_id")
    data = Y_CF_data.merge(data,on="sample_id")
    X = torch.tensor(data.drop(["sample_id","y0","y1","z","y"],axis='columns').values,dtype=torch.float)
    Y = torch.tensor(data[["y"]].values,dtype=torch.float).squeeze(1)
    T = torch.tensor(data[["z"]].values,dtype=torch.float).squeeze(1)
    if null_hypothesis:
        Y_cf = torch.tensor(data[["y0","y1"]].values,dtype=torch.float)
        rand_mat = torch.rand(Y_cf .shape)
        k_th_quant = torch.topk(rand_mat, 1, largest = False)[0][:,-1:]
        mask = rand_mat <= k_th_quant
        Y =  Y_cf[mask]
    perm = torch.randperm(len(Y))
    prop = torch.randn(len(Y)) > 0 
    X_train, Y_train, T_train = X[perm][prop], Y[perm][prop], T[perm][prop]
    X_test, Y_test, T_test = X[perm][~prop], Y[perm][~prop], T[perm][~prop]
    data_train = Data_object(X_train, Y_train, T_train)
    data_test = Data_object(X_test, Y_test, T_test)
    data_full =data_test.join(data_train)
    return data_train, data_test, data_full

In [17]:
X_data = pd.read_csv("data/x.csv")
Y_data = pd.read_csv("data/"+size+"_f.csv")
Y_CF_data = pd.read_csv("data/"+size+"_cf.csv")
data = Y_data.merge(X_data,on="sample_id")
data = Y_CF_data.merge(data,on="sample_id")
X = torch.tensor(data.drop(["sample_id","y0","y1","z","y"],axis='columns').values,dtype=torch.float)
Y = torch.tensor(data[["y"]].values,dtype=torch.float)
T = torch.tensor(data[["z"]].values,dtype=torch.float)

In [22]:
X[T.squeeze(1)==1]

tensor([[ 6.,  3.,  1.,  ..., -1., -1., -1.],
        [ 4.,  4.,  1.,  ..., -1., -1., -1.],
        [ 2.,  3.,  1.,  ..., -1., -1., -1.],
        ...,
        [10.,  6.,  1.,  ..., -1., -1., -1.],
        [10.,  1.,  1.,  ..., -1., -1., -1.],
        [ 9.,  7.,  1.,  ..., -1., -1., -1.]])

In [25]:
LBIDD_data_object(null_hypothesis = False)

(<src.data.Data_object at 0x7fe6c4899a90>,
 <src.data.Data_object at 0x7fe6b2519970>)

In [ ]:
Y_CF_data

,sample_id,z,y
0,2783634,1,11.707886
1,2404025,0,12.994761
2,2483237,1,12.143341
3,2528342,0,13.443833
4,2829307,1,12.914646
...,...,...,...
995,1289626,1,13.035891
996,2689973,1,12.820162
997,2461036,0,12.863620
998,1478202,1,12.099815


In [ ]:
data

,sample_id,z_x,y_x,z_y,y_y,dob_mm,dob_wk,bfacil,ubfacil,bfacil3,...,ca_anen,ca_menin,ca_heart,ca_hernia,ca_ompha,ca_gastro,ca_limb,ca_cleftlp,ca_cleft,ca_hypos
0,2783634,1,11.707886,1,11.707886,6,3,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,2404025,0,12.994761,0,12.994761,7,3,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,2483237,1,12.143341,1,12.143341,4,4,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,2528342,0,13.443833,0,13.443833,8,5,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,2829307,1,12.914646,1,12.914646,2,3,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1289626,1,13.035891,1,13.035891,3,3,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
996,2689973,1,12.820162,1,12.820162,10,6,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
997,2461036,0,12.863620,0,12.863620,2,6,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
998,1478202,1,12.099815,1,12.099815,10,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [ ]:
x = pd.read_csv("data/x.csv")

In [ ]:
x

,sample_id,dob_mm,dob_wk,bfacil,ubfacil,bfacil3,mager41,mager14,mager9,restatus,...,ca_anen,ca_menin,ca_heart,ca_hernia,ca_ompha,ca_gastro,ca_limb,ca_cleftlp,ca_cleft,ca_hypos
0,1481529,11,7,1,1,1,21,8,3,2,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,311677,3,3,1,1,1,25,9,4,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,1225695,11,7,1,1,1,27,9,4,2,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,854512,2,5,1,1,1,31,10,5,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,3023123,7,6,1,1,1,29,9,4,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,978200,9,4,1,1,1,36,11,6,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
99996,1640449,11,4,1,1,1,24,8,3,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
99997,2806154,12,1,1,1,1,23,8,3,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
99998,2983354,6,4,1,1,1,24,8,3,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [ ]:
y.merge(x,on="sample_id")

,sample_id,z,y,dob_mm,dob_wk,bfacil,ubfacil,bfacil3,mager41,mager14,...,ca_anen,ca_menin,ca_heart,ca_hernia,ca_ompha,ca_gastro,ca_limb,ca_cleftlp,ca_cleft,ca_hypos
0,2783634,1,11.707886,6,3,1,1,1,29,9,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,2404025,0,12.994761,7,3,1,1,1,23,8,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,2483237,1,12.143341,4,4,1,1,1,24,8,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,2528342,0,13.443833,8,5,1,1,1,21,8,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,2829307,1,12.914646,2,3,1,1,1,21,8,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1289626,1,13.035891,3,3,1,1,1,27,9,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
996,2689973,1,12.820162,10,6,1,1,1,21,8,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
997,2461036,0,12.863620,2,6,1,1,1,32,10,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
998,1478202,1,12.099815,10,1,1,1,1,33,10,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [ ]:
full_df = pd.read_csv("https://raw.githubusercontent.com/shalit-lab/Benchmarks/master/Twins/Final_data_twins.csv")

In [ ]:
   d = {
            'w': full_df.drop(['T', 'y0', 'y1', 'yf', 'y_cf', 'Propensity'], axis='columns'),
            't': full_df['T'],
            'y': full_df['yf'],
            'y_cf': full_df['y_cf']
        }

In [ ]:
d["w"].values

array([[0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 3.0000e+00,
        3.0000e+00],
       [1.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 1.0000e+00,
        1.0000e+00],
       [2.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 1.0000e+00,
        1.0000e+00],
       ...,
       [1.1981e+04, 0.0000e+00, 0.0000e+00, ..., 1.1000e+01, 2.0000e+00,
        1.0000e+00],
       [1.1982e+04, 0.0000e+00, 0.0000e+00, ..., 1.1000e+01, 3.0000e+00,
        3.0000e+00],
       [1.1983e+04, 0.0000e+00, 1.0000e+00, ..., 6.0000e+00, 1.0000e+00,
        1.0000e+00]])

In [ ]:
def load_twins():
    data = pd.read_csv("https://raw.githubusercontent.com/shalit-lab/Benchmarks/master/Twins/Final_data_twins.csv")
    X = torch.tensor(data.drop(['T', 'y0', 'y1', 'yf', 'y_cf', 'Propensity'],axis='columns').values,)
    Y = torch.tensor(data["yf"],dtype=torch.float)
    T = torch.tensor(data["T"],dtype=torch.float)
    Y_cf = torch.tensor(data["y_cf"],dtype=torch.float)
    return X,Y,T,Y_cf

In [ ]:
load_twins()

AttributeError: 'list' object has no attribute 'values'

In [ ]:
lighter_columns = ['pldel', 'birattnd', 'brstate', 'stoccfipb', 'mager8',
       'ormoth', 'mrace', 'meduc6', 'dmar', 'mplbir', 'mpre5', 'adequacy',
       'orfath', 'frace', 'birmon', 'gestat10', 'csex', 'anemia', 'cardiac',
       'lung', 'diabetes', 'herpes', 'hydra', 'hemo', 'chyper', 'phyper',
       'eclamp', 'incervix', 'pre4000', 'preterm', 'renal', 'rh', 'uterine',
       'othermr', 'tobacco', 'alcohol', 'cigar6', 'drink5', 'crace',
       'data_year', 'nprevistq', 'dfageq', 'feduc6', 'infant_id_0',
       'dlivord_min', 'dtotord_min', 'bord_0',
       'brstate_reg', 'stoccfipb_reg', 'mplbir_reg']
heavier_columns = [ 'pldel', 'birattnd', 'brstate', 'stoccfipb', 'mager8',
       'ormoth', 'mrace', 'meduc6', 'dmar', 'mplbir', 'mpre5', 'adequacy',
       'orfath', 'frace', 'birmon', 'gestat10', 'csex', 'anemia', 'cardiac',
       'lung', 'diabetes', 'herpes', 'hydra', 'hemo', 'chyper', 'phyper',
       'eclamp', 'incervix', 'pre4000', 'preterm', 'renal', 'rh', 'uterine',
       'othermr', 'tobacco', 'alcohol', 'cigar6', 'drink5', 'crace',
       'data_year', 'nprevistq', 'dfageq', 'feduc6',
       'infant_id_1', 'dlivord_min', 'dtotord_min', 'bord_1',
       'brstate_reg', 'stoccfipb_reg', 'mplbir_reg']

In [ ]:
data = []

for i in range(len(t.values)):
    
    #select only if both <=2kg
    if t.iloc[i].values[1]>=2000 or t.iloc[i].values[2]>=2000:
        continue
    
    this_instance_lighter = list(x.iloc[i][lighter_columns].values)
    this_instance_heavier = list(x.iloc[i][heavier_columns].values)
    
    #adding weight
    this_instance_lighter.append(t.iloc[i].values[1])
    this_instance_heavier.append(t.iloc[i].values[2])
    
    #adding treatment, is_heavier
    this_instance_lighter.append(0)
    this_instance_heavier.append(1)
    
    #adding the outcome
    this_instance_lighter.append(y.iloc[i].values[1])
    this_instance_heavier.append(y.iloc[i].values[2])
    data.append(this_instance_lighter)
    data.append(this_instance_heavier)

In [ ]:
cols = [ 'pldel', 'birattnd', 'brstate', 'stoccfipb', 'mager8',
       'ormoth', 'mrace', 'meduc6', 'dmar', 'mplbir', 'mpre5', 'adequacy',
       'orfath', 'frace', 'birmon', 'gestat10', 'csex', 'anemia', 'cardiac',
       'lung', 'diabetes', 'herpes', 'hydra', 'hemo', 'chyper', 'phyper',
       'eclamp', 'incervix', 'pre4000', 'preterm', 'renal', 'rh', 'uterine',
       'othermr', 'tobacco', 'alcohol', 'cigar6', 'drink5', 'crace',
       'data_year', 'nprevistq', 'dfageq', 'feduc6',
       'infant_id', 'dlivord_min', 'dtotord_min', 'bord',
       'brstate_reg', 'stoccfipb_reg', 'mplbir_reg','wt','treatment','outcome']
df = pd.DataFrame(columns=cols,data=data)
df.head()

In [ ]:
df

In [ ]:
df.fillna(value=df.mean(),inplace=True)    #filling the missing values
df.fillna(value=df.mode().loc[0],inplace=True)

data_1 = df[df["treatment"]==1]
data_0 = df[df["treatment"]==0]
print(np.mean(data_1["outcome"]))
print(np.mean(data_0["outcome"]))
print("ATE", np.mean(data_1["outcome"])- np.mean(data_0["outcome"]))

In [ ]:
from rpy2.robjects import r as R

In [ ]:
%R data(lalonde)
%R -o lalonde
lalonde = lalonde.astype({'treat':'int'}, copy=False)

In [ ]:
lalonde

In [ ]:
columns = ["training",   # Treatment assignment indicator
           "age",        # Age of participant
           "education",  # Years of education
           "black",      # Indicate whether individual is black
           "hispanic",   # Indicate whether individual is hispanic
           "married",    # Indicate whether individual is married
           "no_degree",  # Indicate if individual has no high-school diploma
           "re74",       # Real earnings in 1974, prior to study participation
           "re75",       # Real earnings in 1975, prior to study participation
           "re78"]       # Real earnings in 1978, after study end

#treated = pd.read_csv("http://www.nber.org/~rdehejia/data/nswre74_treated.txt", 
#                      delim_whitespace=True, header=None, names=columns)
#control = pd.read_csv("http://www.nber.org/~rdehejia/data/nswre74_control.txt",
#                      delim_whitespace=True, header=None, names=columns)
file_names = ["http://www.nber.org/~rdehejia/data/nswre74_treated.txt",
              "http://www.nber.org/~rdehejia/data/nswre74_control.txt",
              "http://www.nber.org/~rdehejia/data/psid_controls.txt",
              "http://www.nber.org/~rdehejia/data/psid2_controls.txt",
              "http://www.nber.org/~rdehejia/data/psid3_controls.txt",
              "http://www.nber.org/~rdehejia/data/cps_controls.txt",
              "http://www.nber.org/~rdehejia/data/cps2_controls.txt",
              "http://www.nber.org/~rdehejia/data/cps3_controls.txt"]
files = [pd.read_csv(file_name, delim_whitespace=True, header=None, names=columns) for file_name in file_names]
lalonde = pd.concat(files, ignore_index=True)
lalonde = lalonde.sample(frac=1.0, random_state=42)  # Shuffle


In [ ]:
lalonde = lalonde.join((lalonde[["re74", "re75"]] == 0).astype(int), rsuffix=("=0"))


In [ ]:
lalonde

In [ ]:
lalonde = pd.get_dummies(lalonde, columns=["education"], drop_first=True)

In [ ]:
lalonde

In [ ]:
a = lalonde.pop("training")
y = lalonde.pop("re78")
X = lalonde